In [ ]:
# from langchain_community.document_loaders.csv_loader import CSVLoader
import pandas as pd
import re
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from markitdown import MarkItDown
import dotenv
import os

dotenv.load_dotenv()

### 2-1. 프롬프트로 분석하기

In [ ]:
# csv파일 불러오기

df = pd.read_csv("./data/titanic.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
md = MarkItDown()

In [ ]:
convert_df = md.convert("./data/titanic.csv")

In [ ]:
convert_df = convert_df.text_content

In [ ]:
print(convert_df)

In [ ]:
prompt = PromptTemplate.from_template(
    """
    table : {convert}

    당신은 데이터 분석 전문가입니다.
    주어진 table 데이터를 기반으로 데이터 분석에 도움을 주세요.

    query : {query}

    """
    )

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini",
                 temperature=0.,)

In [ ]:
chain = prompt | llm

In [ ]:
query = "이 데이터에서 2번째 행의 이름은 무엇인가요?"

In [ ]:
result = chain.invoke({"convert":convert_df,
                       "query" : query
                       })

In [ ]:
print(result.content)

In [ ]:
result = chain.invoke({"convert":convert_df,
                       "query" : "이 데이터에서 얻을 수 있는 인사이트는 무엇인가요?"
                       })

In [ ]:
print(result.content)

In [ ]:
result = chain.invoke({"convert":df,
                       "query" : "이 데이터를 이용한 분석 순서를 추천해주세요. 무엇부터 해야하나요?"
                       })

In [ ]:
print(result.content)

### 2-1. 도구 사용

In [ ]:
df2 = pd.read_csv("./data/athlete_events.csv")

In [ ]:
df2.info()

In [ ]:
df2.head()

In [ ]:
prompt = ChatPromptTemplate([
    ("system", """당신은 pandas 라이브러리를 사용할 수 있는 전문가입니다. 주어진 df의 형태를 참고하여 코드를 작성합니다. \
                다음은 `print(df.head())`의 형태입니다. {head}
     
                ### 예시1
                ```python
     
                len(df)
                ```
                ### 예시2
                ```python
     
                # sum_gamerounds에 대한 히스토그램
                plt.figure(figsize=(12, 6))
                sns.histplot(df['sum_gamerounds'], bins=10, kde=True)
                plt.title('Distribution of Sum Gamerounds')
                plt.xlabel('Sum Gamerounds')
                plt.ylabel('Frequency')
                plt.grid()
                plt.show()
                ```
     
     """),
    ("human", "{query}")
])

In [ ]:
code_tool = PythonAstREPLTool(name="python_repl_ast", 
                              description="A Python shell. Use this to execute python commands. \
                                           Input should be a valid python command. When using this tool, \
                                           sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.",
                               locals={"df":df2})

In [ ]:
result = code_tool.invoke("for i in range(10): print(i)")

In [ ]:
def extract_code(content):

    content = content.content

    return re.findall(r'```python\n(.*?)\n```', content, re.DOTALL)[0]

In [ ]:
def checker(answer):

    code = extract_code(answer)

    code_prompt = PromptTemplate.from_template(
        """
        아래의 코드를 참고하여 답변합니다.
        절대 코드에 대해 설명하지 마세요.
        독자는 프로그래머가 아닙니다.
        항상 출력되는 값을 기준으로 설명합니다.
        데이터 분석과 관련된 코드가 입력된다면 항상 인사이트를 포함하세요.
        단순한 코드라면 간단하게 대답합니다.

        코드 : {code}

        """)
    

    
    code_chain = code_prompt | llm

    result = code_tool.invoke(code)

    if result == "":
        pass
    else:
        print(result)
    
    response = code_chain.invoke({"code":code})
    print(response.content)
    return response    

In [ ]:
chain = prompt | llm | RunnableLambda(checker)

In [ ]:
result = chain.invoke({"head": df2.head(),
                       "query":"메달 별 획득 비율은 어때?"})

In [ ]:
result = chain.invoke({"head": df2.head(),
                       "query":"출전한 사람은 총 몇명이야?"})